### This Kernel shows 5 different types of Encodings you could use in your project

1. Label Encoding
2. One Hot Encoding
3. Frequency Encoding
4. Leave One Out Encoding
5. Helmert Encoding

We would only use a subset of columns to execute the 5 techniques

In [ ]:
import os
from IPython.display import Image
Image(filename="../input/encodings/encodings.jpg", width= 1200, height=800)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Read the training data

In [ ]:
data = pd.read_csv('/kaggle/input/analytics-vidhya-job-a-thon-may-2021/train_s3TEQDk.csv')

In [ ]:
data.columns =  [_c.upper() for _c in data.columns]

In [ ]:
data.head()

In [ ]:
cols = ['CHANNEL_CODE', 'OCCUPATION']

### 1. Label Encoding

In [ ]:
label_encode_var =  data[cols]

from sklearn.preprocessing import LabelEncoder
for col in cols:
    label_encode_var.loc[:,f'{col}_ENCODED'] = LabelEncoder().fit_transform(label_encode_var[col])

label_encode_var.head()

### 2. One Hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encode_var =  data[cols]
one_hot = OneHotEncoder()

for col in cols:
    enc = one_hot.fit_transform(one_hot_encode_var[col].values.reshape(-1,1)).toarray()
    df_enc = pd.DataFrame(enc, columns = [col + str('_') + str(one_hot.categories_[0][i]) for i in range(len(one_hot.categories_[0]))])
    df_enc = df_enc.iloc[:,1:]
    one_hot_encode_var = pd.concat([one_hot_encode_var, df_enc], axis = 1)
    
one_hot_encode_var.head()

### 3. Frequency Encoding

In [ ]:
freq_encode_var =  data[cols]

for col in cols:
    freq = freq_encode_var.groupby(col).size() / len(freq_encode_var)
    freq_encode_var.loc[:,f'{col}_ENCODED'] = freq_encode_var[col].map(freq)
    
freq_encode_var.head()

### 4. Leave One Out Encoding

In [ ]:
from category_encoders import *

In [ ]:
leave_one_out_encode_var = data[['CHANNEL_CODE', 'OCCUPATION', 'IS_LEAD']]

y = leave_one_out_encode_var.IS_LEAD
X = leave_one_out_encode_var.iloc[:,:-1]
enc = LeaveOneOutEncoder(cols=cols).fit(X, y)
enc_dataset = enc.transform(X)


enc_dataset.columns = [_c.upper() + '_ENCODED' for _c in enc_dataset.columns]
leave_one_out_encode_var = pd.concat([leave_one_out_encode_var,enc_dataset], axis = 1)
leave_one_out_encode_var.head()

### 5. Helmert Encoding

In [ ]:
helmert_encode_var = data[['CHANNEL_CODE', 'OCCUPATION', 'IS_LEAD']]

y = helmert_encode_var.IS_LEAD
X = helmert_encode_var.iloc[:,:-1]
enc = HelmertEncoder(cols=cols,  handle_unknown='value', handle_missing='value').fit(X, y)
enc_dataset = enc.transform(X)


enc_dataset.columns = [_c.upper() + '_ENCODED' for _c in enc_dataset.columns]
helmert_encode_var = pd.concat([helmert_encode_var,enc_dataset], axis = 1)
helmert_encode_var.head()

#### All encoding has their pros and cons which I will cover in a different Kernel. 

### *If found helpful, please leave an UPVOTE 👍*